# <center> IEOR 169 Final Project: Data Generation and Exploration </center>
# <center> Team: Chris Landgrebe, Calvin Suster, Wyatt Walsh </center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Generation" data-toc-modified-id="Data-Generation-1">Data Generation</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Imports-and-Environment-Considerations" data-toc-modified-id="Step-1:-Imports-and-Environment-Considerations-1.1">Step 1: Imports and Environment Considerations</a></span></li><li><span><a href="#Step-2:-Define-Generator-Function" data-toc-modified-id="Step-2:-Define-Generator-Function-1.2">Step 2: Define Generator Function</a></span></li><li><span><a href="#Step-3:-Define-Set-of-Person/Object-Combinations-and-Apply-Generator" data-toc-modified-id="Step-3:-Define-Set-of-Person/Object-Combinations-and-Apply-Generator-1.3">Step 3: Define Set of Person/Object Combinations and Apply Generator</a></span></li></ul></li><li><span><a href="#Model-Output-Exploration-and-Analysis" data-toc-modified-id="Model-Output-Exploration-and-Analysis-2">Model Output Exploration and Analysis</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Load-in-Data-and-View" data-toc-modified-id="Step-1:-Load-in-Data-and-View-2.1">Step 1: Load in Data and View</a></span></li></ul></li></ul></div>

## Data Generation

### Step 1: Imports and Environment Considerations

In [52]:
# import necessary libraries
import numpy as np
import pandas as pd
import os
from IPython.display import display
# set random seed for reproducibility
np.random.seed(18)

### Step 2: Define Generator Function

In [53]:
def synthesize(sizes):
    '''This function takes in row of a dataframe that contains three columns:
    1. the index associated with that row'
    2. the number of objects for this particular dataset
    3. the number of people for this particular dataset
    These parameters are then used to generate a matrix of Uniform[0,1] r.v.s of whic is then massaged into
    a compatible .dat format for use in AMPL'''
    
    # create r.v. matrix of size len(people) by len(objects) then normalize
    ## normalization in this case takes the form of dividing rows by their respective sums
    grid = np.random.uniform(0,1,(sizes[1:]))
    normed = grid/np.sum(grid,axis=0)
    
    # create df with columns whose values correspond with the indice of the value from the original matrix
    df = pd.DataFrame(normed).unstack().reset_index()
    
    # add 1 to all variables so that the ranges start from 1
    df.iloc[:,0:2] = df.iloc[:,0:2] + 1
    
    # move columns for correct AMPL reading
    columns_titles = ['level_1', 'level_0', 0]
    df = df.reindex(columns=columns_titles)
    
    # get index of current row for use in file naming
    index = sizes['index']
    name = './data/generated/' +  str(index) +'.dat'
    [str(i) for i in range(1,500)]
    # generate the sets for people and objects given their size
    people = ' '.join([str(i) for i in range(1,sizes[0]+1)])
    objects = ' '.join([str(i) for i in range(1,sizes[1]+1)])
    
    # since np.savetxt will be used, bundle all other information in the header
    header = 'data; \nset people := %s; \n' % people
    header += 'set objects := %s; \n' % objects
    header += 'param numObjects := %d; \nparam numPeople := %d; \nparam v := ' % (sizes[1],sizes[0])
    
    # save df values space separated under header
    np.savetxt(name,df.values,fmt=['%i','%i', '%f'], header = header, comments = '')
    
    # add final semicolon and close file
    file = open(name, 'a')
    file.write('\n ;')
    file.close()

### Step 3: Define Set of Person/Object Combinations and Apply Generator

In [54]:
# define range of values to combine
## linear spacing is utilized until 160, then a log spacing is used since order of magnitude is more important
values = np.append(np.array([5,10,20,40,80,160]), np.geomspace(320, 5120, 14, dtype = int))

# create df from meshed value ranges.
## must take transpose of meshgrid and reshape in order for output to be .dat file ready
sizes = pd.DataFrame(np.array(np.meshgrid(values,values)).T.reshape(-1,2))

# remove any rows where there are more people than objects since min p will always be 1
sizes = sizes.loc[sizes[0] <= sizes[1],:].reset_index(drop=True).reset_index()

# add 1 to index so that row sets start at 1 and display resultant df
sizes.iloc[:,0] = sizes.iloc[:,0] + 1
display(sizes)

#apply generator function to remaining rows 
# sizes.apply(synthesize, axis = 1) # !!! line 17 is commmented out since the data already exists !!!

,index,0,1
0,1,5,5
1,2,5,10
2,3,5,20
3,4,5,40
4,5,5,80
...,...,...,...
205,206,3342,4136
206,207,3342,5120
207,208,4136,4136
208,209,4136,5120


0      None
1      None
2      None
3      None
4      None
       ... 
205    None
206    None
207    None
208    None
209    None
Length: 210, dtype: object

## Model Output Exploration and Analysis

### Step 1: Load in Data and View

In [34]:
df = pd.read_csv('./data/output/full.txt')
df

,p,solveTime,numObjects,numPeople
0,0.565334,0.202304,5,5
1,0.000000,0.345113,10,5
2,0.000000,0.488031,20,5
3,0.000000,0.560822,40,5
4,0.000000,0.584507,80,5


In [43]:
x = [str(i) for i in range(1,500)]
' '.join(x)

'1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277